In [87]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt

In [49]:
# Transformations that are used in the dataset
trans = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [50]:
# Loading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./', train= True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root='./', train =False, transform=trans)


In [51]:
# We are splitting the training set into a training set and a validation set
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [52]:
# Hyperparameters for the model
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [79]:
# Creating dataloaders for the respective datasets 
train_l = DataLoader ( dataset=  train_dataset, batch_size = batch_size, shuffle= True)
valid_loader = DataLoader(dataset =val_dataset, batch_size = batch_size,   shuffle =False )
test_loader = DataLoader ( dataset = test_dataset, batch_size= batch_size, shuffle = False)

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [64]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(7 * 7 * 64, 1000),
            nn.Linear(1000, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out, 1)  # Flatten the feature tensor
        out = self.classifier(out)
        return out

model = CNN().to(device)


In [69]:
loss_funcn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [70]:
total_steps = len(train_loader)
loss_list = []
accuracy_list = []

In [71]:
for epoch in range (num_epochs ):
    for i, (images, labels) in enumerate(train_l ):
        images= images.to(device )  
        labels =labels.to(device )  

        # Doing the forward pass 
        outputs= model(images ) 
        loss =loss_funcn( outputs, labels )
        loss_list.append(loss.item())

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

       
        # Computing accuracy: 
        
        total= labels.size(0)
        predicted =torch.max(outputs.data, 1)[1]
        correct= ( predicted ==labels).sum().item()
        accuracy_list.append( correct / total)

        if (i + 1) % 100 ==0:
            print('Epoch [{}/{}], Step[{}/{}], Loss:{:.4f }, Accuracy: {:.2f}%'.format( epoch + 1, num_epochs,
                                                                                         i + 1,  total_step,
                                                                                          loss.item(),
                                                                                         ( correct / total ) * 100) )
            
       

Epoch [1/5], Step [100/384], Loss: 0.3527, Accuracy: 89.00%
Epoch [1/5], Step [200/384], Loss: 0.2053, Accuracy: 95.00%
Epoch [1/5], Step [300/384], Loss: 0.3421, Accuracy: 91.00%
Epoch [1/5], Step [400/384], Loss: 0.1521, Accuracy: 94.00%
Epoch [2/5], Step [100/384], Loss: 0.1816, Accuracy: 96.00%
Epoch [2/5], Step [200/384], Loss: 0.2359, Accuracy: 92.00%
Epoch [2/5], Step [300/384], Loss: 0.1966, Accuracy: 90.00%
Epoch [2/5], Step [400/384], Loss: 0.1124, Accuracy: 95.00%
Epoch [3/5], Step [100/384], Loss: 0.1597, Accuracy: 93.00%
Epoch [3/5], Step [200/384], Loss: 0.1478, Accuracy: 96.00%
Epoch [3/5], Step [300/384], Loss: 0.2485, Accuracy: 96.00%
Epoch [3/5], Step [400/384], Loss: 0.2717, Accuracy: 94.00%
Epoch [4/5], Step [100/384], Loss: 0.1584, Accuracy: 95.00%
Epoch [4/5], Step [200/384], Loss: 0.1790, Accuracy: 98.00%
Epoch [4/5], Step [300/384], Loss: 0.1660, Accuracy: 96.00%
Epoch [4/5], Step [400/384], Loss: 0.2048, Accuracy: 93.00%
Epoch [5/5], Step [100/384], Loss: 0.100

In [81]:
# Evaluating the model on the validation set

model.eval()
valid_labels= []

valid_predictions =[]

with torch.no_grad():
    for images, labels in valid_loader:
        
        images= images.to(device)
        labels =labels.to(device )

        outputs = model(images )
        predicted=  torch.max( outputs.data, 1 )[1] 

        valid_labels.extend(labels.cpu().numpy())
        valid_predictions.extend(predicted.cpu().numpy())

valid_accuracy =( sum([1 for i, j in  zip(valid_labels, valid_predictions) if i ==j]) / len(valid_labels) ) * 100

valid_f= f1_score( valid_labels, valid_predictions, average='weighted')

valid_confusion_matrix =confusion_matrix(valid_labels, valid_predictions )

print('Validation Accuracy: {:.2f}%'.format(valid_accuracy))
print('Validation F1 Score: {:.4f}'.format(valid_f1))
print('Validation Confusion Matrix:')
print(valid_confusion_matrix)

Validation Accuracy: 96.99%
Validation F1 Score: 0.9722
Validation Confusion Matrix:
[[1158    0    0    0    1    0    7    0    0    3]
 [   0 1313   15    3    0    1    0    4    3    0]
 [   1    2 1105    3    1    5   24    8    4    0]
 [   1    0   19 1198    0   11    0    5   13    1]
 [   0    1    1    0 1193    0    0    3    3    6]
 [   1    0   26   49    0  979    8   11    2    1]
 [   5    3    5    1    4    6 1152    0    1    0]
 [   0    2    2    4    3    2    0 1242    0    5]
 [   0    3    2    3    1    8    3    2 1182    2]
 [   5    0    1    5   11    5    0   10   10 1117]]


In [84]:
# Evaluating the model on the testing set


model.eval()
testing_labels= []
testing_predictions =[]

with torch.no_grad():
    for images, labels in test_loader:
        images= images.to(device)
        labels =labels.to(device )

        outputs = model(images )
        predicted=  torch.max( outputs.data, 1 )[1] 

        testing_labels.extend(labels.cpu().numpy())
        testing_predictions.extend(predicted.cpu().numpy())


testing_accuracy= (sum([1 for i, j in zip( testing_labels, testing_predictions) if i == j]) / len( testing_labels)) * 100

testing_f1 =f1_score(testing_labels, testing_predictions, average  ='weighted' )

testing_precision= precision_score (testing_labels, testing_predictions, average ='weighted' )

testing_recall =recall_score (testing_labels,testing_predictions, average  ='weighted' )

testing_confusion_matrix= confusion_matrix( testing_labels, testing_predictions)

print('Test Accuracy: {:.2f}%'.format( testing_accuracy ))
print('Test F1 Score: {:.4f}'.format(testing_f1 ) )
print('Test Precision: {:.4f}'.format( testing_precision) )
print('Test Recall: {:.4f}'.format (testing_recall))
print('Test Confusion Matrix:' )
print(testing_confusion_matrix )


Test Accuracy: 97.19%
Test F1 Score: 0.9718
Test Precision: 0.9719
Test Recall: 0.9719
Test Confusion Matrix:
[[ 974    0    0    0    0    1    4    0    0    1]
 [   0 1126    3    0    1    0    1    2    2    0]
 [   1    0  996    1    0    5   16   10    3    0]
 [   1    0    5  979    0   15    0    4    6    0]
 [   0    0    2    0  971    0    0    1    1    7]
 [   1    0   32   37    0  805    6    8    3    0]
 [   9    3    4    0    4    3  933    0    2    0]
 [   1    2    5    1    1    4    0 1011    0    3]
 [   2    1    1    0    0    3    2    2  961    2]
 [   4    2    6    3   12    5    0    9    5  963]]


In [85]:
_PATH = "C:\\Users\\SWETHA"

In [86]:
# Saving the model
torch.save(model.state_dict(), _PATH + '\\conv_net_model.ckpt')